In [23]:
from py4j.java_gateway import JavaGateway

# connect to the java gateway of dl4python
gateway = JavaGateway()


In [24]:
# get a parser from OWL files to DL ontologies
parser = gateway.getOWLParser()

# get a formatter to print in nice DL format
formatter = gateway.getSimpleDLFormatter()


In [25]:
print("Loading the ontology...")

# load an ontology from a file
ontology = parser.parseFile("pizza.owl")

print("Loaded the ontology!")


Loading the ontology...
Loaded the ontology!


In [26]:
print("Converting to binary conjunctions")
gateway.convertToBinaryConjunctions(ontology)


Converting to binary conjunctions


In [27]:
# get the TBox axioms
tbox = ontology.tbox()
axioms = tbox.getAxioms()

print("These are the axioms in the TBox:")
for axiom in axioms:
    print(formatter.format(axiom))


These are the axioms in the TBox:
"PrinceCarlo" ⊑ ∃hasTopping."TomatoTopping"
"MeatTopping" ⊑ "PizzaTopping"
"VegetableTopping" ⊑ "PizzaTopping"
"Napoletana" ⊑ ∃hasTopping."MozzarellaTopping"
"MushroomTopping" ⊑ ∃hasSpiciness."Mild"
"PetitPoisTopping" ⊑ "VegetableTopping"
"SpicyTopping" ≡ (∃hasSpiciness."Hot" ⊓ "PizzaTopping")
"OnionTopping" ⊑ "VegetableTopping"
"American" ⊑ ∃hasCountryOfOrigin.{America}
"GorgonzolaTopping" ⊑ "CheeseTopping"
"Fiorentina" ⊑ ∃hasTopping."MozzarellaTopping"
"Rosa" ⊑ ∀hasTopping.("MozzarellaTopping" ⊔ "TomatoTopping" ⊔ "GorgonzolaTopping")
"Rosa" ⊑ NamedPizza
disjoint("Hot", "Medium", "Mild")
"PineKernelTopping" ⊑ "NutTopping"
"SpinachTopping" ⊑ "VegetableTopping"
domain(hasBase) = "Pizza"
"CaperTopping" ⊑ "VegetableTopping"
"InterestingPizza" ≡ ("Pizza" ⊓ ⩾3hasTopping.⊤)
"VegetarianPizza" ≡ ("Pizza" ⊓ (¬∃hasTopping."MeatTopping" ⊓ ¬∃hasTopping."SeafoodTopping"))
"Rosa" ⊑ ∃hasTopping."TomatoTopping"
"FourSeasons" ⊑ ∃hasTopping."MozzarellaTopping"
⊤ ⊑ ⩽1has

In [28]:
# get all concepts occurring in the ontology
allConcepts = ontology.getSubConcepts()

print()
print("There are ", len(allConcepts), " concepts occurring in the ontology")
print("These are the concepts occurring in the ontology:")
print([formatter.format(x) for x in allConcepts])


There are  227  concepts occurring in the ontology
These are the concepts occurring in the ontology:
['∃hasTopping."CheeseTopping"', '("MozzarellaTopping" ⊔ "TomatoTopping" ⊔ "PrawnsTopping" ⊔ "PeperonataTopping" ⊔ "OnionTopping" ⊔ "TobascoPepperSauceTopping")', '"American"', '"SpicyPizzaEquivalent"', '∀hasTopping.("MozzarellaTopping" ⊔ "MushroomTopping" ⊔ "OliveTopping" ⊔ "LeekTopping" ⊔ "SlicedTomatoTopping" ⊔ "PetitPoisTopping" ⊔ "TomatoTopping" ⊔ "PeperonataTopping")', '∀hasTopping.("MozzarellaTopping" ⊔ "TomatoTopping" ⊔ "AsparagusTopping" ⊔ "ParmezanTopping" ⊔ "HamTopping")', '∀hasTopping.("MozzarellaTopping" ⊔ "TomatoTopping" ⊔ "PrawnsTopping" ⊔ "PeperonataTopping" ⊔ "OnionTopping" ⊔ "TobascoPepperSauceTopping")', '"PizzaBase"', '"NutTopping"', '¬∃hasTopping."SeafoodTopping"', '"PeperonataTopping"', '"FruitTopping"', '"FruttiDiMare"', '"Spiciness"', '"SauceTopping"', '("MozzarellaTopping" ⊔ "MushroomTopping" ⊔ "TomatoTopping")', '∀hasTopping.("MozzarellaTopping" ⊔ "MushroomTopp

In [29]:
# access the type of axioms:
foundGCI = False
foundEquivalenceAxiom = False
print()
print("Looking for axiom types in EL")
for axiom in axioms:
    axiomType = axiom.getClass().getSimpleName() 
    if(not(foundGCI) and axiomType == "GeneralConceptInclusion"):
        print("I found a general concept inclusion:")
        print(formatter.format(axiom))
        print("The left hand side of the axiom is: ", formatter.format(axiom.lhs()))
        print("The right hand side of the axiom is: ", formatter.format(axiom.rhs()))
        print()
        foundGCI = True

    elif(not(foundEquivalenceAxiom) and axiomType == "EquivalenceAxiom"):
        print("I found an equivalence axiom:")
        print(formatter.format(axiom))
        print("The concepts made equivalent are: ")
        for concept in axiom.getConcepts():
            print(" - "+formatter.format(concept))
        print()
        foundEquivalenceAxiom = True



Looking for axiom types in EL
I found a general concept inclusion:
"PrinceCarlo" ⊑ ∃hasTopping."TomatoTopping"
The left hand side of the axiom is:  "PrinceCarlo"
The right hand side of the axiom is:  ∃hasTopping."TomatoTopping"

I found an equivalence axiom:
"SpicyTopping" ≡ (∃hasSpiciness."Hot" ⊓ "PizzaTopping")
The concepts made equivalent are: 
 - "SpicyTopping"
 - (∃hasSpiciness."Hot" ⊓ "PizzaTopping")



In [30]:
# accessing the relevant types of concepts:
foundConceptName=False
foundTop=False
foundExistential=False
foundConjunction=False
foundConceptTypes = set()

print()
print("Looking for concept types in EL")
for concept in allConcepts:
    conceptType = concept.getClass().getSimpleName()
    print(formatter.format(concept))
    
    if(not(conceptType in foundConceptTypes)): 
        print(conceptType)
        foundConceptTypes.add(conceptType)
    if(not(foundConceptName) and conceptType == "ConceptName"):
        print("I found a concept name: "+formatter.format(concept))
        print()
        foundConceptName = True
    elif(not(foundTop) and conceptType == "TopConcept$"):
        print("I found the top concept: "+formatter.format(concept))
        print()
        foundTop = True
    elif(not(foundExistential) and conceptType == "ExistentialRoleRestriction"):
        print("I found an existential role restriction: "+formatter.format(concept))
        print("The role is: "+formatter.format(concept.role()))
        print("The filler is: "+formatter.format(concept.filler()))
        print()
        foundExistential = True
    elif(not(foundConjunction) and conceptType == "ConceptConjunction"):
        print("I found a conjunction: "+formatter.format(concept))
        print("The conjuncts are: ")
        for conjunct in concept.getConjuncts():
            print(" - "+formatter.format(conjunct))
        print()
        foundConjunction=True



Looking for concept types in EL
∃hasTopping."CheeseTopping"
ExistentialRoleRestriction
I found an existential role restriction: ∃hasTopping."CheeseTopping"
The role is: hasTopping
The filler is: "CheeseTopping"

("MozzarellaTopping" ⊔ "TomatoTopping" ⊔ "PrawnsTopping" ⊔ "PeperonataTopping" ⊔ "OnionTopping" ⊔ "TobascoPepperSauceTopping")
ConceptDisjunction
"American"
ConceptName
I found a concept name: "American"

"SpicyPizzaEquivalent"
∀hasTopping.("MozzarellaTopping" ⊔ "MushroomTopping" ⊔ "OliveTopping" ⊔ "LeekTopping" ⊔ "SlicedTomatoTopping" ⊔ "PetitPoisTopping" ⊔ "TomatoTopping" ⊔ "PeperonataTopping")
UniversalRoleRestriction
∀hasTopping.("MozzarellaTopping" ⊔ "TomatoTopping" ⊔ "AsparagusTopping" ⊔ "ParmezanTopping" ⊔ "HamTopping")
∀hasTopping.("MozzarellaTopping" ⊔ "TomatoTopping" ⊔ "PrawnsTopping" ⊔ "PeperonataTopping" ⊔ "OnionTopping" ⊔ "TobascoPepperSauceTopping")
"PizzaBase"
"NutTopping"
¬∃hasTopping."SeafoodTopping"
ConceptComplement
"PeperonataTopping"
"FruitTopping"
"Frutti

In [12]:
# Creating EL concepts and axioms

elFactory = gateway.getELFactory()

conceptA = elFactory.getConceptName("A")
conceptB = elFactory.getConceptName("B")
conjunctionAB = elFactory.getConjunction(conceptA, conceptB)
role = elFactory.getRole("r")
existential = elFactory.getExistentialRoleRestriction(role, conjunctionAB)
top = elFactory.getTop()
conjunction2 = elFactory.getConjunction(top, existential)

gci = elFactory.getGCI(conjunctionAB, conjunction2)

print()
print()
print("I made the following GCI:")
print(formatter.format(gci))
print(top)


In [13]:
# Using the reasoners

elk = gateway.getELKReasoner()
hermit = gateway.getHermiTReasoner() # might the upper case T!

margherita = elFactory.getConceptName('"Margherita"')

print()
print("I am first testing ELK.")
elk.setOntology(ontology)
print()
print("According to ELK, Margherita has the following subsumers: ")
subsumers = elk.getSubsumers(margherita)
for concept in subsumers:
    print(" - ", formatter.format(concept))
print("(", len(subsumers), " in total)")
print()
print("I can also classify the ontology with ELK.")
classificationResult = elk.classify()
print("But I am not printing the result, because that would be too much stuff (it is a dictionary)")
print()

print()
print("I am now testing HermiT.")
hermit.setOntology(ontology)
print()
print("According to HermiT, Margherita has the following subsumers: ")
subsumers = hermit.getSubsumers(margherita)
for concept in subsumers:
    print(" - ", formatter.format(concept))
print("(", len(subsumers), " in total)")
print()
print("I can also classify the ontology with HermiT")
classificationResult = hermit.classify()
print("But I am not printing the result, because that would be too much stuff (it is a dictionary)")
print()


In [ ]:
        # a condition to check if it is one of general concept inclusion or equivalence axioms or it is just a complex class 
        # if it is a complex class,  apply the rules based on the type of the constituents by skipping the initial if condition
        # and apply the rules directly


In [ ]:
from py4j.java_gateway import JavaGateway, GatewayParameters
        
class MyReasoner:
    def __init__(self, ontology):
        self.gateway = JavaGateway(gateway_parameters=GatewayParameters(auto_convert=True))
        self.parser = gateway.getOWLParser()
        self.formatter = gateway.getSimpleDLFormatter()
        print("Loading the ontology...")
        self.ontology = parser.parseFile(ontology)
        print("Loaded the ontology!")
        print("Converting to binary conjunctions")
        self.gateway.convertToBinaryConjunctions(ontology)
        self.tbox = ontology.tbox()
        self.axioms = tbox.getAxioms()
        self.allConcepts = ontology.getSubConcepts()
        self.concepts = ontology.getConcepts()
        self.simple_concepts = ontology.getConceptNames()
        print("The simple concepts are:",self.simple_concepts)
        self.const = gateway.getELFactory()
        self.class_types = ["ConceptConjunction", "ExistentialRoleRestriction"]
        self.individuals = {0:{'concepts': [], 'roles': []}},
        self.axion_types = ["GeneralConceptInclusion", "EquivalenceAxiom"]

    def top_rule(self):
        self.top = elFactory.getTop()
        self.individuals[self.currentIndividual]['concepts'].append(top)
    
    def apply_conjunction_rule1(self, ind, rhs):
        # conjuncts from rhs 
        conjuncts = rhs.getConjuncts()
        
        # iterate over conjunctions 
        for conj in conjuncts:
            # if conjunct not present append 
            if conj not in self.individuals[ind]["concepts"]:
                self.individuals[ind]["concepts"].append(conj)

        

    def apply_conjunction_rule2(self, ind, lhs, rhs):
        # create conjunct concept
        lhs_str = self.formatter.format(lhs)
        rhs_str = self.formatter.format(rhs)
        conjunct = self.const.getConjunction(self.const.getConceptName(lhs_str),self. const.getConceptName(rhs_str))

        # loop over string axioms
        for ax in [self.formatter.format(axiom) for axiom in axioms]:
            # check if cnjunct exits
            if conjunct in ax:
                self.individuals[ind]["concepts"].append(conjunct)
                break


    def r_successor_rule_1(self, rhs, ind):
        # extract roles
        role = formatter.format(rhs)[1:].split()

        # concepts that role point to in ind
        cls = ".".join(role[1:])

        for i, v in self.individuals.items():
            if cls in v["concepts"]:
                self.individuals[ind]["roles"].append((role, i))
                return
        
        # create new individual when successor no individual has cls
        new_ind = max(key for key in self.individuals.keys()) + 1
        self.individuals[new_ind] = {'concepts': [cls], 'roles': []}

        # assign role (successor) to current individual
        self.individuals[ind]["roles"].append(role, new_ind)


        # # all the succesors of ind with role found
        # inds = [self.individuals[ind]["roles"][i][1] for i in range(len(self.individials[ind]["roles"])) if self.individuals[ind]["roles"][i][0] == role[0]]

    def r_successor_rule_2(self, rhs, ind):
        # extract roles
        role = formatter.format(rhs)[1:].split()

        # concepts that role point to in ind
        cls = ".".join(role[1:])
        
        # inidviduals, values 
        for i, v in self.individuals.items():
            if cls in v["concepts"]:
                self.individuals[ind]["concepts"].append(rhs)
                return
        
    def get_classt(self, ind):
        return self.individuals[self.ind]['concepts'].getClass().getSimpleName()
    
    def get_count(self):
        # calculate total amount of concepts and roles in individuals 
        count = sum([len(self.individuals[i]["concepts"]) for i in self.individuals.keys()])
        count += + sum([len(self.individuals[i]["roles"]) for i in self.individuals.keys()])
        return count
    
    def apply_rules(self, rhs, ind):
        if(rhs.getClass().getSimpleName() == self.class_types[0]):
            self.apply_conjunction_rule1(ind, rhs)
            self.apply_conjunction_rule2(ind, rhs)
        else:
            # applying the rules if the lhs is an existential role restriction
            self.r_successor_rule_1(ind, rhs)
            self.r_successor_rule_2(ind, rhs)

    def assign_concepts(self, axiom):
        # get constituents of axiom 
        # if(axiom.getClass().getSimpleName() 
        lhs = axiom.lhs()
        rhs = axiom.rhs()

        # applying equivalence axioms
        if axiom.getClass().getSimpleName() == self.axion_types[1]: 
            for ind, v in self.individuals.items():
                # check lhs in concepts
                if lhs in v["concepts"]:
                    self.individuals[ind]['concepts'].append(rhs)
                    self.apply_rules(rhs, ind)
                # check rhs in concepts 
                if rhs in v["concepts"]:
                    self.individuals[ind]['concepts'].append(lhs)
                    self.apply_rules(lhs, ind)
                    

        # applying general concept inclusion axioms
        else:
            # check if the lhs is a simple concept
            if lhs in self.simple_concepts:
            # apply rules based on the axiom type
                for ind, v in self.individuals.items():
                    # check if lhs occurs inside the individuals concepts
                    if lhs in v["concepts"]:
                        # applying the rules if the lhs is a concept conjunction
                        self.individuals[ind]["concepts"].append(rhs)
                        self.apply_rules(rhs, ind)

                        # if(rhs.getClass().getSimpleName() == self.class_types[0]):
                        #     self.apply_conjunction_rule1(ind, rhs)
                        #     self.apply_conjunction_rule2(ind, rhs)
                        # else:
                        #     # applying the rules if the lhs is an existential role restriction
                        #     self.r_successor_rule_1(ind, rhs)
                        #     self.r_successor_rule_2(ind, rhs)

            # if the lhs is not a simple concept, get the conj
            else:

                # get the conjuncts of the lhs
                lhs_c = lhs.getConjuncts() # [0] = conjunct1, [1] = conjunct2

                # loop over individual
                for ind, v in self.individuals.items():
                    # check if first lhs concepts is in the individuals concepts
                    if lhs_c[0] in v["concepts"]:
                        # check if the lhs is a concept conjunction


                        ## check for simple concepts 
                        if lhs_c[1].getClass().getSimpleName() == self.class_types[0]:
                            if lhs_c[1] in v["concepts"]:
                                self.apply_rules(lhs_c[1], rhs, ind)
                        # check if the lhs is an existential role restriction
                        else:
                            # get the role of the lhs
                            role = formatter.format(lhs)[1:].split(".") # [0] = role, [1] = concept

                            # indfind the individuals that have the role
                            inds = [v["roles"][i][1] for i in range(len(v["roles"])) if v["roles"][i][0] == role[0]]
                            
                            # check if concept is in the found individuals
                            for i in inds:
                                if role[1] in self.individuals[i]["concepts"]:
                                    # append lhs_c[1]
                                    self.individuals[ind]["concepts"].append(lhs_c[1])
                                    # append lhs
                                    self.individuals[ind]["concepts"].append(lhs)
                                    # => rhs is present 
                                    self.individuals[ind]["concepts"].append(rhs)
                                    # apply rules to rhs 
                                    self.apply_rules(lhs, rhs, ind)
                                    break
                    
            # if(axiom.getClass().getSimpleName() == self.axion_types[0]:
               
        
    #     if lhs in concepts:
    #     else:
    #         self.individuals[self.currentIndividual].append(lhs)

    def get_subsumer(self, class_name):

        self.individuals[self.currentIndividual].append(class_name)
        self.top_rule()
        if self.get_classt(self.currentIndividual) in self.class_types:
            self.apply_conjunction_rule1(self.currentIndividual, self.get_classt(self.currentIndividual))
            self.r_successor_rule(self.currentIndividual, self.get_classt(self.currentIndividual))

        # convergence criterian
        change = True

        current_count = self.get_count()

        while change:

            # previous count  
            for axiom in axioms:
                self.assign_concepts(axiom)
            
            # get new count 
            new_count = self.get_count()

            # check for convergence
            if new_count == current_count:
                change = False
            else:
                current_count = new_count 
        
        # return subsumer of given class
        return self.individuals[0]["concepts"]
        

In [22]:
elk = MyReasoner("pizza.owl")
